In [1]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general
import scqubits as sc


In [2]:
#Validating T2 Calcs Against SCQUBITS

EJ = torch.tensor(12, requires_grad=True, dtype=torch.double)
EC = torch.tensor(2.6, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.2, requires_grad=True, dtype=torch.double)
dim = 110

fluxonium_auto = Fluxonium(EJ, EC, EL, flux, dim, 'auto_H')
fluxonium_create = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')
fluxonium_sc = sc.Fluxonium(EJ = EJ.item(), EC = EC.item(), EL = EL.item(), flux = flux.item(),cutoff = dim)

print((fluxonium_sc.t1_quasiparticle_tunneling()*general.effective_t1_rate(fluxonium_auto, "t1_quasiparticle_tunneling").item()))
print((fluxonium_sc.t1_charge_impedance()*general.effective_t1_rate(fluxonium_auto , "t1_charge_impedance").item()))
print((fluxonium_sc.t1_capacitive()*general.effective_t1_rate(fluxonium_auto, "t1_capacitive").item()))
print((fluxonium_sc.t1_inductive() * general.effective_t1_rate(fluxonium_auto, "t1_inductive").item()))
print((fluxonium_sc.t1_flux_bias_line() * general.effective_t1_rate(fluxonium_auto , "t1_flux_bias_line").item()))
print((fluxonium_sc.tphi_1_over_f_cc() * general.effective_tphi_rate(fluxonium_auto , "tphi_1_over_f_cc").item()))
print((fluxonium_sc.tphi_1_over_f_flux() * general.effective_tphi_rate(fluxonium_auto, "tphi_1_over_f_flux()").item()))



See documentation for details.
This warning can be disabled by executing:
scqubits.settings.T1_DEFAULT_WARNING=False

 /Users/judd/Documents/optimisation_of_superconduncting_circuits/venv/lib/python3.8/site-packages/scqubits/core/noise.py: 1195UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3575.)
 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/general.py: 73

0.9999999499081519
0.999999998487069
0.9999999806933993
0.9999998471125674
0.9999998388498631
0.9999997816073528
1.0000000055992981


In [3]:
general.t2_rate(fluxonium_create, fluxonium_create.t1_supported_noise_channels(), fluxonium_create.tphi_supported_noise_channels())


tensor([[0.0027]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [4]:
#Optimisation

#random values
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

# PARAMETERS

dim = 110
learn_rate = 3e4

# BOUNDS

EJ_bounds = [2.5, 150]
EC_bounds = [1e-3, 8]
EL_bounds = [2e-1, 10]
flux_bounds = [0,1]

# RECORDING VALUES
EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
cost_vals = []

# GRADIENT DESCENT

max_T2 = 0

for i in range(0,700):

    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

    #wasted computational time doing the .tphi_supported
    T2 = general.t2_rate(fluxonium , fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

    if T2 > max_T2:
        max_T2 = T2
    
    #autograd on objective function
    cost = 1/T2
    cost.backward()

    cost_vals.append(cost.item())

    print("CYCLE: ",i)
    print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
    print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())
    print("change EJ,EC,EL,flux = ",EJ.grad.item()*learn_rate, EC.grad.item()*learn_rate, EL.grad.item()*learn_rate,flux.grad.item()*learn_rate)
    
    #update parameters according to gradient
    with torch.no_grad():
        # Near the boundary, we project the gradient onto it. This is the typical approach to constraints.
        if EJ - EJ.grad*learn_rate > EJ_bounds[0] and EJ.data - EJ.grad*learn_rate < EJ_bounds[1]:
            EJ -= EJ.grad*learn_rate
        elif EJ - EJ.grad*learn_rate <= EJ_bounds[0]:
            EJ += (torch.tensor(EJ_bounds[0]) - EJ)
        else:
            EJ += (torch.tensor(EJ_bounds[1]) - EJ)
        EJ_vals.append(EJ.item())
        
        if EC - EC.grad*learn_rate > EC_bounds[0] and EC - EC.grad*learn_rate < EC_bounds[1]:
            EC -= EC.grad*learn_rate
        elif EC - EC.grad*learn_rate <= EC_bounds[0]:
            EC += (torch.tensor(EC_bounds[0]) - EC)
        else:
            EC += (torch.tensor(EC_bounds[1]) - EC)
        EC_vals.append(EC.item())
        
        if EL - EL.grad*learn_rate > EL_bounds[0] and EL - EL.grad*learn_rate < EL_bounds[1]:
            EL -= EL.grad*learn_rate
        elif EL - EL.grad*learn_rate <= EL_bounds[0]:
            EL += (torch.tensor(EL_bounds[0]) - EL)
        else:
            EL += (torch.tensor(EL_bounds[1]) - EL)
        EL_vals.append(EL.item())

    EJ.grad = None
    EC.grad = None
    EL.grad = None
    flux.grad = None

#append last cost value to cost array
fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')


T2 = general.t2_rate(fluxonium , fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

cost = 1/T2
print("CYCLE: END")
print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())

cost_vals.append(cost.item())


CYCLE:  0
T =  0.8265727518115477 , cost =  1.2098148624042624 , Best T =  0.8265727518115477
EJ,EC,EL,flux =  5.57025769932855 5.666738231416506 8.240552015846303 0.5
change EJ,EC,EL,flux =  3083.5873084896225 -971.4759625512919 -5821.048181858945 297.57229366292773
CYCLE:  1
T =  1.3325546715976453 , cost =  0.7504382531645518 , Best T =  1.3325546715976453
EJ,EC,EL,flux =  2.5 8.0 10.0 0.5
change EJ,EC,EL,flux =  1289.8358837155504 -131.57441339121263 -2468.740526149184 36.987755379412846
CYCLE:  2
T =  1.3325546715976453 , cost =  0.7504382531645518 , Best T =  1.3325546715976453
EJ,EC,EL,flux =  2.5 8.0 10.0 0.5
change EJ,EC,EL,flux =  1289.8358837155504 -131.57441339121263 -2468.740526149184 36.987755379412846
CYCLE:  3
T =  1.3325546715976453 , cost =  0.7504382531645518 , Best T =  1.3325546715976453
EJ,EC,EL,flux =  2.5 8.0 10.0 0.5
change EJ,EC,EL,flux =  1289.8358837155504 -131.57441339121263 -2468.740526149184 36.987755379412846
CYCLE:  4
T =  1.3325546715976453 , cost =  0